# Loading the Data

In [15]:
# load data

# import the experiment utilities package
import exputils as eu
import numpy as np

# create an experiment data loader, by default it will load data from '../experiments'
experiment_data_loader = eu.gui.jupyter.ExperimentDataLoaderWidget()

display(experiment_data_loader)
experiment_data_loader.load_data()

/Users/dajo/miniconda3/envs/urbf/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning:

The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.

/Users/dajo/miniconda3/envs/urbf/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning:

The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.



ExperimentDataLoaderWidget(children=(Box(children=(Button(description='Update Descriptions', layout=Layout(hei…

/Users/dajo/miniconda3/envs/urbf/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning:

The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.



Output()

# Check for Data Completeness

In [5]:
import dill
from pmlb import fetch_data

experiment_data_loader.experiment_data["000001"]["repetition_data"][0].keys()

data_dir = "../experiments/experiment_000001/repetition_000000/data/"

with open(data_dir+"dataset.dill", 'rb') as data:
    dataset_names = dill.load(data)

print(dataset_names)

for dataset_name in dataset_names:
    X,Y = fetch_data(dataset_name, return_X_y=True)
    print(f"{dataset_name}: x -> {X.shape}")

['1027_ESL', '1029_LEV', '1030_ERA', '1096_FacultySalaries', '192_vineyard', '210_cloud', '228_elusage', '485_analcatdata_vehicle', '519_vinnie', '523_analcatdata_neavote', '529_pollen', '556_analcatdata_apnea2', '557_analcatdata_apnea1', '579_fri_c0_250_5', '594_fri_c2_100_5', '596_fri_c2_250_5', '597_fri_c2_500_5', '599_fri_c2_1000_5', '601_fri_c1_250_5', '609_fri_c0_1000_5', '611_fri_c3_100_5', '612_fri_c1_1000_5', '613_fri_c3_250_5', '617_fri_c3_500_5', '624_fri_c0_100_5', '628_fri_c3_1000_5', '631_fri_c1_500_5', '649_fri_c0_500_5', '656_fri_c1_100_5', '663_rabe_266', '678_visualizing_environmental', '687_sleuth_ex1605', '690_visualizing_galaxy', '712_chscase_geyser1', 'banana']
1027_ESL: x -> (488, 4)
1029_LEV: x -> (1000, 4)
1030_ERA: x -> (1000, 4)
1096_FacultySalaries: x -> (50, 4)
192_vineyard: x -> (52, 2)
210_cloud: x -> (108, 5)
228_elusage: x -> (55, 2)
485_analcatdata_vehicle: x -> (48, 4)
519_vinnie: x -> (380, 2)
523_analcatdata_neavote: x -> (100, 2)
529_pollen: x -> (

# Plotting the Data

In [2]:
## experiment data selection plotter that takes as input the data loader to plot its loaded data
experiment_data_plotter = eu.gui.jupyter.ExperimentDataPlotSelectionWidget(experiment_data_loader)
display(experiment_data_plotter)

/Users/dajo/miniconda3/envs/urbf/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/Users/dajo/miniconda3/envs/urbf/lib/python3.11/site-packages/traitlets/traitlets.py:1387: DeprecationWarning: Passing unrecognized arguments to super(CodeProducerWidget).__init__(cildren=[]).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(


ExperimentDataPlotSelectionWidget(children=(HBox(children=(Label(value='Data Sources:', layout=Layout(min_widt…

In [7]:
### Execution time


In [16]:
import os
import dill

def load_experiment_data(experiment_num):
    num_str = "%06d" % experiment_num
    #repetition_str = "%06d" % repetition_num


    base_dir = "../experiments/experiment_"+num_str+"/"
    data_dir = "../experiments/experiment_"+num_str+"/repetition_000000/data/"

    means = {}
    stds = {}

    train_loss = {}
    test_loss = {}
    val_loss = {}
    
    print(f"Loading {data_dir}")
    
    dataset_file = data_dir+"dataset.dill"
    
    if not os.path.isfile(dataset_file):
        return means, stds, train_loss, test_loss, val_loss
    
    with open(dataset_file, 'rb') as data:
        datasets = dill.load(data)
        
        
    # Iterate over each entry in the given path
    for repetition_entry in os.listdir(base_dir):
        # Create full path
        repetition_path = os.path.join(base_dir, repetition_entry)

        if not os.path.isdir(repetition_path):
            continue


        for dataset_entry in os.listdir(repetition_path):
            dataset_path = os.path.join(repetition_path, dataset_entry)
            if os.path.isdir(dataset_path) and "data" in dataset_entry and dataset_entry != "data":
                dataset_name = dataset_entry[4:]

                if dataset_name not in means:
                    means[dataset_name] = []
                    stds[dataset_name] = []
                    train_loss[dataset_name] = []
                    test_loss[dataset_name] = []
                    val_loss[dataset_name] = []

                _rep_dataset_means = []
                _rep_dataset_stds = []

                _test_loss = None
                _train_loss = None
                _val_loss = None

                for data_entry in os.listdir(dataset_path):
                
                    try:
                        data_path = os.path.join(dataset_path, data_entry)

                        if "mean" in data_entry and data_entry[0] != ".":
                            mean = np.load(data_path)
                            _rep_dataset_means.append(mean)

                        if "std" in data_entry and data_entry[0] != ".":
                            std = np.load(data_path)
                            _rep_dataset_stds.append(std)

                        if "train_loss" in data_entry and data_entry[0] != ".":
                            _train_loss = np.load(data_path)

                        if "test_loss" in data_entry and data_entry[0] != ".":
                            _test_loss = np.load(data_path)

                        if "val_loss" in data_entry and data_entry[0] != ".":
                            _val_loss = np.load(data_path)
                    except:
                        print(f"Error at retrieving data for: {data_entry}")

                if len(_rep_dataset_means) > 0:
                    _rep_dataset_means = np.stack(_rep_dataset_means)

                if len(_rep_dataset_stds) > 0:
                    _rep_dataset_stds = np.stack(_rep_dataset_stds)

                means[dataset_name].append(_rep_dataset_means)
                stds[dataset_name].append(_rep_dataset_stds)
                train_loss[dataset_name].append(_train_loss)
                test_loss[dataset_name].append(_test_loss)
                val_loss[dataset_name].append(_val_loss)


    for key,val in means.items():
        try:
            means[key] = np.stack(val).transpose((0,2,1))
            #print(f"loaded means {means[key].shape}")
        except:
            pass
            #print("Not present")

    for key,val in stds.items():
        try:
            stds[key] = np.stack(val).transpose((0,2,1))
            #print(f"loaded stds {stds[key].shape}")
        except:
            pass
            #print("Not present")

    for key,val in train_loss.items():
        try:
            train_loss[key] = np.stack(val).transpose((1,0))
            #print(f"loaded train {train_loss[key].shape}")
        except:
            pass
            #print("Not present")
    for key,val in test_loss.items():
        try:
            test_loss[key] = np.stack(val).transpose((1,0))
            #print(f"loaded test {test_loss[key].shape}")
        except:
            pass
            #print("Not present")
    for key,val in val_loss.items():
        try:
            val_loss[key] = np.stack(val).transpose((1,0))
            #print(f"loaded val {val_loss[key].shape}")
        except:
            pass
            #print("Not present")

            
    return means, stds, train_loss, test_loss, val_loss
    
means, stds, train_loss, test_loss, val_loss = load_experiment_data(15)   



Loading ../experiments/experiment_000015/repetition_000000/data/


In [4]:
dataset_name = "1096_FacultySalaries"

## Mean value

In [5]:
## plotting
import numpy as np
import plotly.graph_objects as go




data = means[dataset_name][0]

print(data.shape)

# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []
for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))

# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()



KeyError: '1096_FacultySalaries'

## StandardDeviation Values

In [ ]:
### plot
import numpy as np
import plotly.graph_objects as go


data = stds[dataset_name][0]
    
# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []
for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))

# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()

In [ ]:
### plot
import numpy as np
import plotly.graph_objects as go


data = test_loss[dataset_name]
    
# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []


data = np.expand_dims(data.mean(axis = 1), axis = 1)

for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))
    


# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()

In [ ]:
### plot
import numpy as np
import plotly.graph_objects as go


data = val_loss[dataset_name]
    
# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []


data = np.expand_dims(data.mean(axis = 1), axis = 1)

for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))
    


# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import traceback 

max_method_num = 36

min_test_loss = {}
min_val_test_loss = {}

for i in range(1,max_method_num):
    means, stds, train_loss, test_loss, val_loss = load_experiment_data(i)    
    for dataset in list(train_loss.keys()):
        print(f"Analyzing: {dataset} {i}")
        
        try:
            rep_min_test_loss = np.min(np.mean(test_loss[dataset],axis = 1),axis=0)
            idx_min_val_loss = np.argmin(np.mean(val_loss[dataset],axis = 1),axis=0)
            
            if dataset not in min_test_loss:
                min_test_loss[dataset] = {}
                
            if dataset not in min_val_test_loss:
                min_val_test_loss[dataset] = {}

            min_test_loss[dataset][i] = rep_min_test_loss
            min_val_test_loss[dataset][i] = np.mean(test_loss[dataset],axis = 1)[idx_min_val_loss]
            
            print(np.mean(test_loss[dataset],axis = 1)[idx_min_val_loss])
        except: 
            traceback.print_exc() 

### remove all inclompete datasets...
to_delete = []
for key, val in min_test_loss.items():
    if len(val.keys()) < max_method_num - 1:
        print(f"For {key} not all datasets present {len(val.keys())}")
        to_delete.append(key)


#for key in to_delete:
#    del min_test_loss[key]
#    del min_val_test_loss[key]
        
### Calculate with reference to specific method
reference_method = 3

# Calculate relative min test loss
relative_min_test_loss = {}
relative_min_val_test_loss = {}

for dataset_name, dataset_vals in min_test_loss.items():
    relative_min_test_loss[dataset_name] = {method: val / dataset_vals[reference_method] for method, val in dataset_vals.items()}

for dataset_name, dataset_vals in min_val_test_loss.items():    
    relative_min_val_test_loss[dataset_name] = {method: val / dataset_vals[reference_method] for method, val in dataset_vals.items()}    


#full_relative_min_test_loss = {}
        

Loading ../experiments/experiment_000001/repetition_000000/data/
Analyzing: analcatdata_aids 1
0.4233747625189651
Analyzing: bupa 1
0.2635780909513112
Analyzing: phoneme 1
0.11610104508537504
Analyzing: confidence 1
2.4610753926393114
Analyzing: balance_scale 1
0.20503163786795806
Analyzing: analcatdata_boxing1 1
0.2623111552851547
Analyzing: analcatdata_lawsuit 1
0.025587715100045164
Analyzing: irish 1
0.17894636719101437
Analyzing: parity5 1
0.6634173695027418
Analyzing: haberman 1
0.20773786161273797
Analyzing: hayes_roth 1
0.3006848949857972
Analyzing: new_thyroid 1
0.28554057151791123
Analyzing: prnn_synth 1
0.08497409764528752
Analyzing: analcatdata_happiness 1
0.565857390765886
Analyzing: analcatdata_germangss 1
1.4829361128123708
Analyzing: analcatdata_boxing2 1
0.29193997594652665
Analyzing: iris 1
0.04066142155844446
Analyzing: analcatdata_asbestos 1
0.3152686826801793
Analyzing: tae 1
0.8631106372072019
Analyzing: lupus 1
0.19210994153739394
Analyzing: analcatdata_dmft 1
2.9

Analyzing: analcatdata_aids 14
0.7090437412261963
Analyzing: confidence 14
0.7585372030735016
Analyzing: balance_scale 14
0.042346421629190445
Analyzing: analcatdata_boxing1 14
0.5438656657934189
Analyzing: analcatdata_lawsuit 14
0.06221693055704236
Analyzing: analcatdata_happiness 14
1.1116713136434555
Analyzing: analcatdata_germangss 14
1.1775074899196625
Analyzing: analcatdata_boxing2 14
0.6852906942367554
Analyzing: analcatdata_asbestos 14
0.6953005343675613
Analyzing: analcatdata_dmft 14
3.598228842020035
Loading ../experiments/experiment_000015/repetition_000000/data/
Analyzing: analcatdata_aids 15
0.7102196216583252
Analyzing: bupa 15
0.6768147349357605
Analyzing: confidence 15
1.0540897697210312
Analyzing: balance_scale 15
0.07058417052030563
Analyzing: analcatdata_boxing1 15
0.5268571227788925
Analyzing: analcatdata_lawsuit 15
0.05411573266610503
Analyzing: analcatdata_happiness 15
1.0866465419530869
Analyzing: analcatdata_germangss 15
1.302894949913025
Analyzing: analcatdata_

Analyzing: analcatdata_aids 31
0.7064633965492249
Analyzing: bupa 31
0.7022138983011246
Analyzing: confidence 31
1.8198406398296356
Analyzing: balance_scale 31
0.03940192796289921
Analyzing: analcatdata_boxing1 31
0.5325144529342651
Analyzing: analcatdata_lawsuit 31
0.20798031240701675
Analyzing: analcatdata_happiness 31
1.1263695657253265
Analyzing: analcatdata_germangss 31
1.3973738253116608
Analyzing: analcatdata_boxing2 31
0.5820751339197159
Analyzing: analcatdata_asbestos 31
0.6980639696121216
Analyzing: analcatdata_dmft 31
3.5652164816856384
Loading ../experiments/experiment_000032/repetition_000000/data/
Analyzing: analcatdata_aids 32
0.7109008729457855
Analyzing: bupa 32
0.6992409080266953
Analyzing: confidence 32
1.8207045793533325
Analyzing: balance_scale 32
0.0230735931545496
Analyzing: analcatdata_boxing1 32
0.45490018278360367
Analyzing: analcatdata_lawsuit 32
0.2205851972103119
Analyzing: analcatdata_happiness 32
1.1222299337387085
Analyzing: analcatdata_germangss 32
1.38

In [18]:
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.Plotly

colors = [*colors,*colors,*colors,*colors]

dataset_names = list(min_test_loss.keys())

for dataset_name in dataset_names:
    # Extract methods and corresponding test losses for the current dataset
    methods = list(relative_min_test_loss[dataset_name].keys())
    test_losses = [relative_min_test_loss[dataset_name][method] for method in methods]

    # Create a bar chart for the current dataset
    fig = go.Figure(data=[
        go.Bar(
            x=methods,
            y=test_losses,
            name=dataset_name,
            marker_color=colors[:len(methods)]  # Assign different colors to each bar
        )
    ])

    print(methods)
    
    # Customize the layout
    fig.update_layout(
        title=f'Relative Min Test Loss by Method for {dataset_name}',
        xaxis=dict(
            title='Method',
            #tickmode='array',
            #tickvals=list(range(len(methods))),
            #ticktext=methods
        ),
        yaxis=dict(
            title='Relative Min Test Loss'
        ),
        legend_title=dataset_name
        # Additional customizations can be added here
    )

    # Show the plot
    fig.show()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


[1, 2, 3]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


...

[1, 2, 3]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


[1, 2, 3]


[1, 2, 3]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


In [19]:

dataset_names = list(min_val_test_loss.keys())

for dataset_name in dataset_names:
    # Preparing data for bar chart
    methods = list(relative_min_val_test_loss[dataset_name].keys())
    test_losses = [relative_min_val_test_loss[dataset_name][method] for method in methods]


    # Create a bar chart for the current dataset
    fig = go.Figure(data=[
        go.Bar(
            x=methods,
            y=test_losses,
            name=dataset_name,
            marker_color=colors[:len(methods)]  # Assign different colors to each bar
        )
    ])

    # Customize the layout
    fig.update_layout(
        title=f'Relative Min Test Loss by Method for {dataset_name}',
        xaxis=dict(
            title='Method',
        ),
        yaxis=dict(
            title='Relative Min Test Loss'
        ),
        legend_title=dataset_name
        # Additional customizations can be added here
    )

    # Show the plot
    fig.show()
    

In [20]:
### The mean over all datasets is not a good indicator!! (Prone to outliers) Every dataset has different properties and therefore different methods perform best.

import plotly.express as px

dataset_names = list(min_val_test_loss.keys())

summary_test_loss = {}


for dataset_name, dataset_vals in relative_min_val_test_loss.items():
    for method_name, method_val in dataset_vals.items():
        if method_name not in summary_test_loss:
            summary_test_loss[method_name] = []
        
        summary_test_loss[method_name].append(method_val)
        
for method_name, method_vals in summary_test_loss.items():
    summary_test_loss[method_name] = np.mean(method_vals)


# Adding bars for each dataset
methods = list(summary_test_loss.keys())
index = np.arange(len(methods))  # the label locations

test_losses = [summary_test_loss[method] for method in methods]
dataset_name = "summary"
# Create a bar chart for the current dataset
fig = go.Figure(data=[
    go.Bar(
        x=methods,
        y=test_losses,
        name=dataset_name,
        marker_color = colors[:len(methods)]  # Assign different colors to each bar
    )
])

# Customize the layout
fig.update_layout(
    title=f'Relative Min Test Loss by Method for {dataset_name}',
    xaxis=dict(
        title='Method',
        #tickmode='array',
        tickvals=list(range(len(methods))),
        ticktext=methods
    ),
    yaxis=dict(
        title='Relative Min Test Loss'
    ),
    legend_title=dataset_name
    # Additional customizations can be added here
)

# Show the plot
fig.show()


In [21]:
import plotly.graph_objects as go
import numpy as np

dataset_names = list(relative_min_val_test_loss.keys())
methods = list(set([method for dataset in relative_min_val_test_loss.values() for method in dataset.keys()]))

# Prepare data for heatmap
heatmap_data = []
best_methods_indices = []

for dataset in dataset_names:
    dataset_vals = []
    best_loss = float('inf')
    best_method_index = -1

    for index, method in enumerate(methods):
        loss = relative_min_val_test_loss[dataset].get(method, None)
        
        if loss is not None: ### only used to get a clearer contrast between low value scores..
            loss = min(100,loss)
        
        dataset_vals.append(loss)

        if loss is not None and loss < best_loss:
            best_loss = loss
            best_method_index = index
    
    heatmap_data.append(dataset_vals)
    best_methods_indices.append(best_method_index)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=methods,
    y=dataset_names,
    colorscale='Turbo'
))

# Overlay with scatter plot to highlight the best methods
for i, best_index in enumerate(best_methods_indices):
    fig.add_trace(go.Scatter(
        x=[methods[best_index]],
        y=[dataset_names[i]],
        mode='markers',
        marker=dict(
            color='red',
            size=4,
            line=dict(
                color='red',
                width=1
                
            )
        )
    ))

# Customize layout
fig.update_layout(
    title='Relative Min Test Loss by Method and Dataset',
    xaxis=dict(title='Method'),
    yaxis=dict(title='Dataset'),
    showlegend=False
)

# Show the plot
fig.show()


In [25]:
import plotly.graph_objects as go

# Assuming 'relative_min_val_test_loss' and 'dataset_names' are defined as before
dataset_names = list(relative_min_val_test_loss.keys())

# Finding methods across all datasets
methods = list(set([method for dataset_vals in relative_min_val_test_loss.values() for method in dataset_vals.keys()]))

# Initialize x, y, z for 3D scatter plot locations and test losses for color scale
x, y, z, test_losses = [], [], [], []

for i, dataset in enumerate(dataset_names):
    for j, method in enumerate(methods):
        loss = relative_min_val_test_loss[dataset].get(method)
        if loss is not None:
            x.append(j)  # method index
            y.append(i)  # dataset index
            z.append(loss)  # test loss
            test_losses.append(loss)

# Create 3D scatter plot with a continuous color scale
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=test_losses,  # Assign color based on test loss
        colorscale='Viridis',  # Choose a color scale
        colorbar=dict(title='Test Loss'),
        opacity=0.8
    )
)])

# Customize layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title='Method', tickvals=list(range(len(methods))), ticktext=methods),
        yaxis=dict(title='Dataset', tickvals=list(range(len(dataset_names))), ticktext=dataset_names),
        zaxis=dict(title='Relative Min Test Loss')
    ),
    title='3D Scatter Plot of Relative Min Test Loss by Method and Dataset'
)

# Show the plot
fig.show()


